# SynCirc: Framework for Efficient Secure Circuit Synthesis from High-Level Languages

## Setup

The following cell will set up an environment for experimenting with our SynCirc toolchain. On Colab, the setup takes about 5 minutes.

The code will download binaries for XLS, Yosys, and the Zig compiler toolchain as well as source code from our work. SynCirc is compiled by Zig from source on-the-fly. While Yosys and Zig are fetched from their upstream binary distributions, we modified the XLS codebase and thus provide our own binaries.

In [1]:
# @title
import os
import pathlib

yosys_version = '0.38_93_g84116c9a3'

# download repository
!curl --show-error -L -o repo.zip https://github.com/encryptogroup/SynCirc/archive/refs/heads/main.zip
!unzip -q -o repo.zip
!mv SynCirc-main/* .

# set up XLS
!curl --show-error -L -o xls-bin.tar.xz 'https://github.com/encryptogroup/FLUENT/releases/download/v1.0.0/xls-bin.tar.xz'
!tar xJf xls-bin.tar.xz
xls_bin_path = pathlib.Path('bazel-bin')
xlscc_path = xls_bin_path / 'xls' / 'contrib' / 'xlscc' / 'xlscc'
xls_opt_path = xls_bin_path / 'xls' / 'tools' / 'opt_main'
xls_codegen_path = xls_bin_path / 'xls' / 'tools' / 'codegen_main'
XLSCC = str(xlscc_path.resolve())
XLS_OPT = str(xls_opt_path.resolve())
XLS_CODEGEN = str(xls_codegen_path.resolve())
%env XLSCC={XLSCC}
%env XLS_OPT={XLS_OPT}
%env XLS_CODEGEN={XLS_CODEGEN}

# set up Yosys
!curl -L -O https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -p conda-env/
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
!conda-env/bin/conda install -yq -c "litex-hub" yosys={yosys_version}
conda_bin_path = conda_prefix_path / 'bin'
os.environ['PATH'] += os.pathsep + str(conda_bin_path.resolve())

# set up SynCirc
!curl --show-error -L https://ziglang.org/download/0.10.0/zig-linux-x86_64-0.10.0.tar.xz | tar xJ
!zig-linux-x86_64-0.10.0/zig build
fresh_bin_path = pathlib.Path('zig-out') / 'bin'
os.environ['PATH'] += os.pathsep + str(fresh_bin_path.resolve())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 16.9M    0 16.9M    0     0   9.7M      0 --:--:--  0:00:01 --:--:-- 20.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  227M  100  227M    0     0  81.7M      0  0:00:02  0:00:02 --:--:-- 96.2M
env: XLSCC=/content/bazel-bin/xls/contrib/xlscc/xlscc
env: XLS_OPT=/content/bazel-bin/xls/tools/opt_main
env: XLS_CODEGEN=/content/bazel-bin/xls/tools/codegen_main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0   100M      0 

## Examples

First, we will have a quick look at some simple examples.

### Verilog

The following Verilog code implements a simple 32-bit integer adder.

In [ ]:
%%writefile example_verilog.v

module adder (input wire [31:0] a, input wire [31:0] b, output wire [31:0] c);
  assign c = a + b;
endmodule

Writing paper_example_verilog.v


In [ ]:
!syncirc example_verilog.v -t adder
%pfile example_verilog.syncirc

compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_verilog.spfe` not found.


Let's have a look at the circuit depth and size.

In [ ]:
!stats example_verilog.syncirc

### LUT circuits

In the previous step, we generated circuits consisting of AND, AND3, AND4, XOR, and INV gates. Let's try generating a circuit consisting of 3-input LUTs.

In [ ]:
!syncirc example_verilog.v -t adder -L 3
%pfile example_verilog.syncirc

In [ ]:
!stats example_verilog.syncirc

### C/C++

This is the same example implemented in C, made possible with High-Level Synthesis (HLS).

In [ ]:
%%writefile example_c.c

int adder(int a, int b) {
  return a + b;
}

Writing paper_example_c.c


In [ ]:
!syncirc example_c.c -t adder
%pfile example_c.syncirc

running high-level synthesis with XLS... compiling public components... compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_c.spfe` not found.


In [ ]:
!stats example_c.syncirc